# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [1]:
# Setup
from __future__ import division, print_function, unicode_literals

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [2]:
import keras as kr

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


# Convolutional Neural Network

## Import Dataset

In [3]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [4]:
def to_one_hot(X):
    n_values = np.max(X) + 1
    y = np.eye(n_values)[X[:,0]]
    return y

y_train_adj = to_one_hot(y_train)
y_test_adj  = to_one_hot(y_test)

In [5]:
print(np.shape(x_train))

(50000, 32, 32, 3)


## Coarse Training

** Defining Structure**

In [6]:
def indep_layers(X):
    net = Conv2D(64, (4, 4), strides=(1, 1), padding='same', activation='elu')(X)
    net = Conv2D(64, (4, 4), strides=(1, 1), padding='same', activation='elu')(net)
    net = MaxPooling2D((2, 2), padding='valid')(net)
    net = Conv2D(128, (2, 2), strides=(1, 1), padding='same', activation='elu')(net)
    net = Conv2D(128, (2, 2), strides=(1, 1), padding='same', activation='elu')(net)
    net = Conv2D(128, (2, 2), strides=(1, 1), padding='same', activation='elu')(net)
    net = Flatten()(net)

    net = Dense(512, activation='elu')(net)
    net = Dropout(0.50)(net)
    net = Dense(256, activation='elu')(net)
    net = Dropout(0.50)(net)
    net = Dense(100, activation='softmax')(net)
    
    return net

In [7]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, Multiply, Add, Dot, RepeatVector, Reshape
from keras.optimizers import SGD
from keras.models import Model

coarse_categories = 20
fine_categories = 100
sigma = .01

input_img = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

## SHARED LAYERS
net_shared = Conv2D(16, (4, 4), strides=(1, 1), padding='same', activation='elu')(input_img)
net_shared = Conv2D(16, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)
net_shared = Conv2D(32, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = Conv2D(32, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)
net_shared = Conv2D(64, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)

# COARSE CLASSIFIER
# Get the coarse predictions from fine
output_c = indep_layers(net_shared)

base_model = Model(inputs=input_img, outputs=output_c)

base_model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
base_model.fit(x_train, y_train_adj, initial_epoch=300, epochs=305, batch_size=256, validation_split=.1)
base_model.save_weights('data/models/base_model_'+str(500))

Train on 45000 samples, validate on 5000 samples
Epoch 301/305
45000/45000 [==============================] - 53s - loss: 2.8182 - acc: 0.2865 - val_loss: 2.6733 - val_acc: 0.3262
Epoch 302/305
45000/45000 [==============================] - 52s - loss: 2.8161 - acc: 0.2840 - val_loss: 2.6456 - val_acc: 0.3364
Epoch 303/305
45000/45000 [==============================] - 54s - loss: 2.8175 - acc: 0.2864 - val_loss: 2.6579 - val_acc: 0.3222
Epoch 304/305
45000/45000 [==============================] - 53s - loss: 2.8071 - acc: 0.2874 - val_loss: 2.6468 - val_acc: 0.3330
Epoch 305/305
45000/45000 [==============================] - 53s - loss: 2.8011 - acc: 0.2871 - val_loss: 2.8266 - val_acc: 0.3052


## Fine-Tuning

**Define New Layers**

In [11]:
from __future__ import absolute_import

from keras.layers import initializers, regularizers, constraints
from keras.engine.topology import Layer, InputSpec
from keras import backend as K
from keras.legacy import interfaces

class Threshold(Layer):
    """Thresholded Rectified Linear Unit.
    It follows:
    `f(x) = 1 for x > theta`,
    `f(x) = 0 otherwise`.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as the input.
    # Arguments
        theta: float >= 0. Threshold value.
    """

    def __init__(self, theta=.5, **kwargs):
        super(Threshold, self).__init__(**kwargs)
        self.theta = K.cast_to_floatx(theta)

    def call(self, inputs, mask=None):
        return K.cast(K.greater(inputs, self.theta), K.floatx())

    def get_config(self):
        config = {'theta': float(self.theta)}
        base_config = super(Threshold, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [12]:
def calc_inputs():
    return [Input(shape=([100]), dtype='float32', name='c0'), \
        Input(shape=([100]), dtype='float32', name='c1'), \
        Input(shape=([100]), dtype='float32', name='c2'), \
        Input(shape=([100]), dtype='float32', name='c3'), \
        Input(shape=([100]), dtype='float32', name='c4'), \
        Input(shape=([100]), dtype='float32', name='c5'), \
        Input(shape=([100]), dtype='float32', name='c6'), \
        Input(shape=([100]), dtype='float32', name='c7'), \
        Input(shape=([100]), dtype='float32', name='c8'), \
        Input(shape=([100]), dtype='float32', name='c9'), \
        Input(shape=([100]), dtype='float32', name='c10'), \
        Input(shape=([100]), dtype='float32', name='c11'), \
        Input(shape=([100]), dtype='float32', name='c12'), \
        Input(shape=([100]), dtype='float32', name='c13'), \
        Input(shape=([100]), dtype='float32', name='c14'), \
        Input(shape=([100]), dtype='float32', name='c15'), \
        Input(shape=([100]), dtype='float32', name='c16'), \
        Input(shape=([100]), dtype='float32', name='c17'), \
        Input(shape=([100]), dtype='float32', name='c18'), \
        Input(shape=([100]), dtype='float32', name='c19')]

In [13]:
def calc_prob_ave(fine_pred, fine_to_coarse):
    coarse_pred = Dot(1)([fine_pred,fine_to_coarse])
    coarse_pred_adj = RepeatVector(100)(coarse_pred)
    coarse_pred_adj = Flatten()(coarse_pred_adj)
    prob_ave = Multiply()([fine_to_coarse, coarse_pred_adj])

    return prob_ave

def calc_threshold(fine_pred, fine_to_coarse, shared_layer):
    coarse_pred = Dot(1)([fine_pred, fine_to_coarse])
    coarse_pred_thresholded = Threshold()(coarse_pred)
    threshold_adj = RepeatVector(12*8*8)(coarse_pred_thresholded)
    threshold_adj = Reshape((8, 8, 12))(threshold_adj)
    shared_layer_adj = Multiply()([shared_layer, threshold_adj])
    
    return shared_layer_adj

In [14]:
c = calc_inputs()

for i in range(20):
    thresholded = calc_threshold(base_model.output, c[i], net_shared)
    fine_output = indep_layers(thresholded)

    prob_ave = calc_prob_ave(output_c, c[i])

    weighted = Multiply()([fine_output, prob_ave])
    
    if i==0: output_f = weighted
    else: output_f = Add()([weighted, output_f])

model = Model(inputs=[base_model.input, c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8], c[9], c[10], c[11], c[12], c[13], c[14], c[15], c[16], c[17], c[18], c[19]], 
              outputs=output_f)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Load Coarse-To-Fine Mappings**

In [15]:
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

#np.set_printoptions(threshold=np.inf)
#print(C2K_dot)

**Fit Training Data**

In [21]:
ck = np.zeros((20, len(x_train), len(C2K_dot[0])))
for i in range(len(x_train)):
    for j in range(20):
        ck[j,i] = C2K_dot[j]

index= 0
step = 25
stop = 500

while index < stop:
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, initial_epoch=index, epochs=index+step, batch_size=256, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))
    index += step

Train on 45000 samples, validate on 5000 samples
Epoch 1/25
45000/45000 [==============================] - 115s - loss: 3.2674 - acc: 0.1848 - val_loss: 3.2203 - val_acc: 0.2022
Epoch 2/25
45000/45000 [==============================] - 115s - loss: 3.2589 - acc: 0.1871 - val_loss: 3.2067 - val_acc: 0.2062
Epoch 3/25
45000/45000 [==============================] - 127s - loss: 3.2670 - acc: 0.1842 - val_loss: 3.2162 - val_acc: 0.2004
Epoch 4/25
45000/45000 [==============================] - 140s - loss: 3.2599 - acc: 0.1856 - val_loss: 3.2282 - val_acc: 0.2012
Epoch 5/25
45000/45000 [==============================] - 136s - loss: 3.2634 - acc: 0.1881 - val_loss: 3.2270 - val_acc: 0.2066
Epoch 6/25
45000/45000 [==============================] - 134s - loss: 3.2521 - acc: 0.1887 - val_loss: 3.2242 - val_acc: 0.2006
Epoch 7/25
45000/45000 [==============================] - 124s - loss: 3.2586 - acc: 0.1861 - val_loss: 3.2202 - val_acc: 0.2054
Epoch 8/25
45000/45000 [========================

45000/45000 [==============================] - 114s - loss: 3.2550 - acc: 0.1961 - val_loss: 3.2607 - val_acc: 0.2022
Epoch 63/75
45000/45000 [==============================] - 114s - loss: 3.2658 - acc: 0.1932 - val_loss: 3.2409 - val_acc: 0.2084
Epoch 64/75
45000/45000 [==============================] - 114s - loss: 3.2616 - acc: 0.1945 - val_loss: 3.2365 - val_acc: 0.2066
Epoch 65/75
45000/45000 [==============================] - 114s - loss: 3.2597 - acc: 0.1935 - val_loss: 3.2409 - val_acc: 0.2026
Epoch 66/75
45000/45000 [==============================] - 114s - loss: 3.2671 - acc: 0.1921 - val_loss: 3.2439 - val_acc: 0.2066
Epoch 67/75
45000/45000 [==============================] - 116s - loss: 3.2633 - acc: 0.1952 - val_loss: 3.2467 - val_acc: 0.2084
Epoch 68/75
45000/45000 [==============================] - 119s - loss: 3.2567 - acc: 0.1934 - val_loss: 3.2470 - val_acc: 0.2046
Epoch 69/75
45000/45000 [==============================] - 119s - loss: 3.2604 - acc: 0.1940 - val_los

45000/45000 [==============================] - 119s - loss: 3.2410 - acc: 0.2004 - val_loss: 3.2507 - val_acc: 0.2054
Epoch 124/125
45000/45000 [==============================] - 119s - loss: 3.2419 - acc: 0.2022 - val_loss: 3.2438 - val_acc: 0.2066
Epoch 125/125
45000/45000 [==============================] - 119s - loss: 3.2590 - acc: 0.1972 - val_loss: 3.2374 - val_acc: 0.2010
Train on 45000 samples, validate on 5000 samples
Epoch 126/150
45000/45000 [==============================] - 119s - loss: 3.2463 - acc: 0.2006 - val_loss: 3.2518 - val_acc: 0.2084
Epoch 127/150
45000/45000 [==============================] - 119s - loss: 3.2415 - acc: 0.2011 - val_loss: 3.2502 - val_acc: 0.2124
Epoch 128/150
45000/45000 [==============================] - 119s - loss: 3.2456 - acc: 0.2014 - val_loss: 3.2565 - val_acc: 0.2116
Epoch 129/150
45000/45000 [==============================] - 119s - loss: 3.2433 - acc: 0.2021 - val_loss: 3.2577 - val_acc: 0.2052
Epoch 130/150
45000/45000 [==============

45000/45000 [==============================] - 119s - loss: 3.2436 - acc: 0.2026 - val_loss: 3.2410 - val_acc: 0.2092
Epoch 185/200
45000/45000 [==============================] - 119s - loss: 3.2296 - acc: 0.2074 - val_loss: 3.2500 - val_acc: 0.2064
Epoch 186/200
45000/45000 [==============================] - 119s - loss: 3.2359 - acc: 0.2032 - val_loss: 3.2528 - val_acc: 0.2090
Epoch 187/200
45000/45000 [==============================] - 119s - loss: 3.2224 - acc: 0.2064 - val_loss: 3.2513 - val_acc: 0.2124
Epoch 188/200
45000/45000 [==============================] - 119s - loss: 3.2494 - acc: 0.2006 - val_loss: 3.2459 - val_acc: 0.2074
Epoch 189/200
45000/45000 [==============================] - 119s - loss: 3.2362 - acc: 0.2048 - val_loss: 3.2531 - val_acc: 0.2036
Epoch 190/200
45000/45000 [==============================] - 119s - loss: 3.2398 - acc: 0.2042 - val_loss: 3.2564 - val_acc: 0.2126
Epoch 191/200
45000/45000 [==============================] - 119s - loss: 3.2395 - acc: 0.

45000/45000 [==============================] - 119s - loss: 3.2327 - acc: 0.2052 - val_loss: 3.2617 - val_acc: 0.2120
Epoch 246/250
45000/45000 [==============================] - 120s - loss: 3.2257 - acc: 0.2065 - val_loss: 3.2695 - val_acc: 0.2028
Epoch 247/250
45000/45000 [==============================] - 119s - loss: 3.2333 - acc: 0.2049 - val_loss: 3.2631 - val_acc: 0.2070
Epoch 248/250
45000/45000 [==============================] - 120s - loss: 3.2262 - acc: 0.2055 - val_loss: 3.2564 - val_acc: 0.2058
Epoch 249/250
45000/45000 [==============================] - 119s - loss: 3.2205 - acc: 0.2077 - val_loss: 3.2562 - val_acc: 0.2086
Epoch 250/250
45000/45000 [==============================] - 119s - loss: 3.2209 - acc: 0.2065 - val_loss: 3.2641 - val_acc: 0.2122
Train on 45000 samples, validate on 5000 samples
Epoch 251/275
45000/45000 [==============================] - 119s - loss: 3.2272 - acc: 0.2053 - val_loss: 3.2675 - val_acc: 0.2106
Epoch 252/275
45000/45000 [==============

45000/45000 [==============================] - 119s - loss: 3.2219 - acc: 0.2087 - val_loss: 3.2510 - val_acc: 0.2010
Epoch 307/325
45000/45000 [==============================] - 119s - loss: 3.2270 - acc: 0.2063 - val_loss: 3.2466 - val_acc: 0.2114
Epoch 308/325
45000/45000 [==============================] - 119s - loss: 3.2289 - acc: 0.2079 - val_loss: 3.2659 - val_acc: 0.2106
Epoch 309/325
45000/45000 [==============================] - 119s - loss: 3.2268 - acc: 0.2080 - val_loss: 3.2528 - val_acc: 0.2146
Epoch 310/325
45000/45000 [==============================] - 120s - loss: 3.2230 - acc: 0.2077 - val_loss: 3.2559 - val_acc: 0.2094
Epoch 311/325
45000/45000 [==============================] - 119s - loss: 3.2155 - acc: 0.2087 - val_loss: 3.2703 - val_acc: 0.2020
Epoch 312/325
45000/45000 [==============================] - 119s - loss: 3.2146 - acc: 0.2078 - val_loss: 3.2760 - val_acc: 0.2094
Epoch 313/325
45000/45000 [==============================] - 119s - loss: 3.2108 - acc: 0.

45000/45000 [==============================] - 119s - loss: 3.2099 - acc: 0.2131 - val_loss: 3.2801 - val_acc: 0.2108
Epoch 368/375
45000/45000 [==============================] - 119s - loss: 3.2173 - acc: 0.2083 - val_loss: 3.2623 - val_acc: 0.2154
Epoch 369/375
45000/45000 [==============================] - 119s - loss: 3.2195 - acc: 0.2100 - val_loss: 3.2953 - val_acc: 0.2088
Epoch 370/375
45000/45000 [==============================] - 119s - loss: 3.2192 - acc: 0.2081 - val_loss: 3.2602 - val_acc: 0.2042
Epoch 371/375
45000/45000 [==============================] - 120s - loss: 3.2205 - acc: 0.2084 - val_loss: 3.2781 - val_acc: 0.2076
Epoch 372/375
45000/45000 [==============================] - 119s - loss: 3.2068 - acc: 0.2111 - val_loss: 3.2735 - val_acc: 0.2096
Epoch 373/375
45000/45000 [==============================] - 119s - loss: 3.2119 - acc: 0.2106 - val_loss: 3.2610 - val_acc: 0.2096
Epoch 374/375
45000/45000 [==============================] - 119s - loss: 3.2186 - acc: 0.

45000/45000 [==============================] - 115s - loss: 3.2131 - acc: 0.2106 - val_loss: 3.2622 - val_acc: 0.2100
Epoch 429/450
45000/45000 [==============================] - 115s - loss: 3.2174 - acc: 0.2130 - val_loss: 3.2677 - val_acc: 0.2114
Epoch 430/450
45000/45000 [==============================] - 119s - loss: 3.2174 - acc: 0.2107 - val_loss: 3.2725 - val_acc: 0.2036
Epoch 431/450
45000/45000 [==============================] - 120s - loss: 3.2200 - acc: 0.2105 - val_loss: 3.2739 - val_acc: 0.2070
Epoch 432/450
45000/45000 [==============================] - 114s - loss: 3.2191 - acc: 0.2121 - val_loss: 3.2765 - val_acc: 0.2028
Epoch 433/450
45000/45000 [==============================] - 119s - loss: 3.2156 - acc: 0.2101 - val_loss: 3.2817 - val_acc: 0.2118
Epoch 434/450
45000/45000 [==============================] - 120s - loss: 3.2121 - acc: 0.2123 - val_loss: 3.2773 - val_acc: 0.2092
Epoch 435/450
45000/45000 [==============================] - 122s - loss: 3.2059 - acc: 0.

45000/45000 [==============================] - 114s - loss: 3.2064 - acc: 0.2107 - val_loss: 3.2668 - val_acc: 0.2148
Epoch 490/500
45000/45000 [==============================] - 114s - loss: 3.1985 - acc: 0.2171 - val_loss: 3.2713 - val_acc: 0.2102
Epoch 491/500
45000/45000 [==============================] - 114s - loss: 3.1955 - acc: 0.2127 - val_loss: 3.2720 - val_acc: 0.2054
Epoch 492/500
45000/45000 [==============================] - 114s - loss: 3.2063 - acc: 0.2120 - val_loss: 3.2741 - val_acc: 0.2082
Epoch 493/500
45000/45000 [==============================] - 114s - loss: 3.2132 - acc: 0.2120 - val_loss: 3.2938 - val_acc: 0.2080
Epoch 494/500
45000/45000 [==============================] - 114s - loss: 3.2095 - acc: 0.2131 - val_loss: 3.2632 - val_acc: 0.2080
Epoch 495/500
45000/45000 [==============================] - 114s - loss: 3.2047 - acc: 0.2126 - val_loss: 3.2666 - val_acc: 0.2102
Epoch 496/500
45000/45000 [==============================] - 114s - loss: 3.2111 - acc: 0.

In [22]:
stop = 1000

while index < stop:
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, initial_epoch=index, epochs=index+step, batch_size=256, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))
    index += step

Train on 45000 samples, validate on 5000 samples
Epoch 501/525
45000/45000 [==============================] - 114s - loss: 3.2000 - acc: 0.2125 - val_loss: 3.2802 - val_acc: 0.2080
Epoch 502/525
45000/45000 [==============================] - 114s - loss: 3.2061 - acc: 0.2118 - val_loss: 3.2820 - val_acc: 0.2010
Epoch 503/525
45000/45000 [==============================] - 114s - loss: 3.2023 - acc: 0.2124 - val_loss: 3.2853 - val_acc: 0.2134
Epoch 504/525
45000/45000 [==============================] - 114s - loss: 3.2072 - acc: 0.2127 - val_loss: 3.2686 - val_acc: 0.2096
Epoch 505/525
45000/45000 [==============================] - 114s - loss: 3.2044 - acc: 0.2118 - val_loss: 3.2741 - val_acc: 0.2072
Epoch 506/525
45000/45000 [==============================] - 114s - loss: 3.2072 - acc: 0.2138 - val_loss: 3.2839 - val_acc: 0.2068
Epoch 507/525
45000/45000 [==============================] - 114s - loss: 3.2076 - acc: 0.2132 - val_loss: 3.2716 - val_acc: 0.2108
Epoch 508/525
45000/45000 [

45000/45000 [==============================] - 119s - loss: 3.2062 - acc: 0.2124 - val_loss: 3.2800 - val_acc: 0.2056
Epoch 562/575
45000/45000 [==============================] - 120s - loss: 3.1945 - acc: 0.2152 - val_loss: 3.2812 - val_acc: 0.2124
Epoch 563/575
45000/45000 [==============================] - 119s - loss: 3.2051 - acc: 0.2141 - val_loss: 3.2826 - val_acc: 0.2064
Epoch 564/575
45000/45000 [==============================] - 119s - loss: 3.2140 - acc: 0.2136 - val_loss: 3.2786 - val_acc: 0.2066
Epoch 565/575
45000/45000 [==============================] - 119s - loss: 3.2112 - acc: 0.2121 - val_loss: 3.2680 - val_acc: 0.2078
Epoch 566/575
45000/45000 [==============================] - 119s - loss: 3.2042 - acc: 0.2137 - val_loss: 3.2823 - val_acc: 0.2068
Epoch 567/575
45000/45000 [==============================] - 119s - loss: 3.2102 - acc: 0.2117 - val_loss: 3.2816 - val_acc: 0.2140
Epoch 568/575
45000/45000 [==============================] - 119s - loss: 3.2124 - acc: 0.

45000/45000 [==============================] - 119s - loss: 3.2087 - acc: 0.2154 - val_loss: 3.2758 - val_acc: 0.2054
Epoch 623/625
45000/45000 [==============================] - 120s - loss: 3.2145 - acc: 0.2134 - val_loss: 3.2757 - val_acc: 0.2150
Epoch 624/625
45000/45000 [==============================] - 120s - loss: 3.2095 - acc: 0.2104 - val_loss: 3.2735 - val_acc: 0.2144
Epoch 625/625
45000/45000 [==============================] - 119s - loss: 3.1989 - acc: 0.2140 - val_loss: 3.2813 - val_acc: 0.2076
Train on 45000 samples, validate on 5000 samples
Epoch 626/650
45000/45000 [==============================] - 120s - loss: 3.1985 - acc: 0.2163 - val_loss: 3.2745 - val_acc: 0.2068
Epoch 627/650
45000/45000 [==============================] - 119s - loss: 3.1978 - acc: 0.2122 - val_loss: 3.2706 - val_acc: 0.2070
Epoch 628/650
45000/45000 [==============================] - 119s - loss: 3.1871 - acc: 0.2171 - val_loss: 3.2649 - val_acc: 0.2062
Epoch 629/650
45000/45000 [==============

45000/45000 [==============================] - 119s - loss: 3.2023 - acc: 0.2162 - val_loss: 3.2646 - val_acc: 0.2068
Epoch 684/700
45000/45000 [==============================] - 120s - loss: 3.2127 - acc: 0.2140 - val_loss: 3.2646 - val_acc: 0.2126
Epoch 685/700
45000/45000 [==============================] - 119s - loss: 3.2029 - acc: 0.2149 - val_loss: 3.2740 - val_acc: 0.2056
Epoch 686/700
45000/45000 [==============================] - 119s - loss: 3.2042 - acc: 0.2117 - val_loss: 3.2954 - val_acc: 0.2108
Epoch 687/700
45000/45000 [==============================] - 119s - loss: 3.2084 - acc: 0.2129 - val_loss: 3.2829 - val_acc: 0.2114
Epoch 688/700
45000/45000 [==============================] - 119s - loss: 3.1949 - acc: 0.2164 - val_loss: 3.2790 - val_acc: 0.2110
Epoch 689/700
45000/45000 [==============================] - 119s - loss: 3.2122 - acc: 0.2145 - val_loss: 3.2669 - val_acc: 0.2168
Epoch 690/700
45000/45000 [==============================] - 119s - loss: 3.2089 - acc: 0.

45000/45000 [==============================] - 115s - loss: 3.2075 - acc: 0.2152 - val_loss: 3.2674 - val_acc: 0.2110
Epoch 745/750
45000/45000 [==============================] - 116s - loss: 3.2142 - acc: 0.2124 - val_loss: 3.2781 - val_acc: 0.2182
Epoch 746/750
45000/45000 [==============================] - 123s - loss: 3.1973 - acc: 0.2162 - val_loss: 3.2697 - val_acc: 0.2080
Epoch 747/750
45000/45000 [==============================] - 122s - loss: 3.2103 - acc: 0.2147 - val_loss: 3.3005 - val_acc: 0.2132
Epoch 748/750
45000/45000 [==============================] - 123s - loss: 3.2096 - acc: 0.2128 - val_loss: 3.2772 - val_acc: 0.2074
Epoch 749/750
45000/45000 [==============================] - 127s - loss: 3.2032 - acc: 0.2140 - val_loss: 3.2840 - val_acc: 0.2128
Epoch 750/750
45000/45000 [==============================] - 125s - loss: 3.2058 - acc: 0.2166 - val_loss: 3.2755 - val_acc: 0.2128
Train on 45000 samples, validate on 5000 samples
Epoch 751/775
28672/45000 [==============

KeyboardInterrupt: 

**Fit on dataset delayed (for use in browsers)**

This is a complex network and the dataset is not small either so this may occasionally crash if run in a browser. 

In the case that it does, use this function to incrementally train the network 5 (# can be changed if necessary), save the model and then reload it for the next increment of training.

In [68]:
while index < 300:
    model.load_weights('data/models/layer_'+str(index), by_name=False)
    index += step
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, epochs=index+step, batch_size=64, initial_epoch=index, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 26/50
45000/45000 [==============================] - 170s - loss: 4.6786 - acc: 0.0108 - val_loss: 4.6034 - val_acc: 0.0122
Epoch 27/50
45000/45000 [==============================] - 171s - loss: 4.6714 - acc: 0.0107 - val_loss: 4.6022 - val_acc: 0.0090
Epoch 28/50
45000/45000 [==============================] - 170s - loss: 4.6674 - acc: 0.0106 - val_loss: 4.6029 - val_acc: 0.0112
Epoch 29/50
45000/45000 [==============================] - 169s - loss: 4.6663 - acc: 0.0106 - val_loss: 4.6007 - val_acc: 0.0092
Epoch 30/50
45000/45000 [==============================] - 169s - loss: 4.6614 - acc: 0.0109 - val_loss: 4.6021 - val_acc: 0.0086
Epoch 31/50
45000/45000 [==============================] - 175s - loss: 4.6592 - acc: 0.0105 - val_loss: 4.5997 - val_acc: 0.0130
Epoch 32/50
45000/45000 [==============================] - 172s - loss: 4.6592 - acc: 0.0110 - val_loss: 4.5989 - val_acc: 0.0130
Epoch 33/50
45000/45000 [================

45000/45000 [==============================] - 175s - loss: 4.4657 - acc: 0.0214 - val_loss: 4.4111 - val_acc: 0.0220
Epoch 149/150
45000/45000 [==============================] - 175s - loss: 4.4614 - acc: 0.0204 - val_loss: 4.4022 - val_acc: 0.0188
Epoch 150/150
45000/45000 [==============================] - 175s - loss: 4.4643 - acc: 0.0199 - val_loss: 4.4131 - val_acc: 0.0180
Train on 45000 samples, validate on 5000 samples
Epoch 151/175
45000/45000 [==============================] - 174s - loss: 4.4628 - acc: 0.0210 - val_loss: 4.4010 - val_acc: 0.0224
Epoch 152/175
45000/45000 [==============================] - 175s - loss: 4.4589 - acc: 0.0207 - val_loss: 4.4013 - val_acc: 0.0190
Epoch 153/175
45000/45000 [==============================] - 174s - loss: 4.4594 - acc: 0.0214 - val_loss: 4.4043 - val_acc: 0.0186
Epoch 154/175
45000/45000 [==============================] - 176s - loss: 4.4565 - acc: 0.0214 - val_loss: 4.4328 - val_acc: 0.0180
Epoch 155/175
45000/45000 [==============

45000/45000 [==============================] - 176s - loss: 4.3981 - acc: 0.0242 - val_loss: 4.4539 - val_acc: 0.0122
Epoch 210/225
45000/45000 [==============================] - 174s - loss: 4.3937 - acc: 0.0241 - val_loss: 4.3866 - val_acc: 0.0170
Epoch 211/225
45000/45000 [==============================] - 175s - loss: 4.3916 - acc: 0.0240 - val_loss: 4.3505 - val_acc: 0.0232
Epoch 212/225
45000/45000 [==============================] - 175s - loss: 4.3927 - acc: 0.0250 - val_loss: 4.3570 - val_acc: 0.0208
Epoch 213/225
45000/45000 [==============================] - 175s - loss: 4.3907 - acc: 0.0246 - val_loss: 4.3454 - val_acc: 0.0224
Epoch 214/225
45000/45000 [==============================] - 175s - loss: 4.3913 - acc: 0.0227 - val_loss: 4.3433 - val_acc: 0.0226
Epoch 215/225
45000/45000 [==============================] - 175s - loss: 4.3914 - acc: 0.0245 - val_loss: 4.3622 - val_acc: 0.0244
Epoch 216/225
45000/45000 [==============================] - 175s - loss: 4.3888 - acc: 0.

In [76]:
coarse_layers = ['conv2d_327', 'dropout_417', 'conv2d_328', 'dropout_418', 'conv2d_329', 'flatten_154', 'dense_271', 'dropout_419', 'dense_272', 'dropout_420']

model.load_weights('data/models/layer_300', 'conv2d_327', 'conv2d_328')

TypeError: load_weights() takes from 2 to 3 positional arguments but 4 were given

In [69]:
while index < 500:
    model.load_weights('data/models/layer_'+str(index), by_name=False)
    index += step
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, epochs=index+step, batch_size=64, initial_epoch=index, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 326/350
45000/45000 [==============================] - 175s - loss: 4.2875 - acc: 0.0311 - val_loss: 4.3052 - val_acc: 0.0236
Epoch 327/350
45000/45000 [==============================] - 175s - loss: 4.2863 - acc: 0.0305 - val_loss: 4.2905 - val_acc: 0.0272
Epoch 328/350
45000/45000 [==============================] - 175s - loss: 4.2870 - acc: 0.0313 - val_loss: 4.3172 - val_acc: 0.0240
Epoch 329/350
45000/45000 [==============================] - 176s - loss: 4.2857 - acc: 0.0305 - val_loss: 4.3033 - val_acc: 0.0244
Epoch 330/350
45000/45000 [==============================] - 172s - loss: 4.2854 - acc: 0.0319 - val_loss: 4.3024 - val_acc: 0.0260
Epoch 331/350
45000/45000 [==============================] - 175s - loss: 4.2866 - acc: 0.0304 - val_loss: 4.3346 - val_acc: 0.0210
Epoch 332/350
40896/45000 [==========================>...] - ETA: 15s - loss: 4.2837 - acc: 0.0307

KeyboardInterrupt: 

**Evaluate on testing set**

In [ ]:
ct = np.zeros((20,len(x_test),100))
for i in range(len(x_test)):
    for j in range(20):
        ct[j,i] = C2K_dot[j]
        
model.evaluate([x_test, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
          y_test_adj, batch_size=64, verbose=1)